In [1]:
import os
import pandas as pd
import gradio as gr

In [ ]:
file_word = pd.read_excel("doc/speaker_wordlist.xls", sheet_name="Word_filename")
# file_word

fw, wf = {}, {}
for _, row in file_word.iterrows():
    code = row["FILE NAME"]
    word = row["WORD"]
    wf[word] = code
    fw[code] = word

In [4]:
noisered_path = "/data/zaki/UASpeech/audio/noisereduce"
participants = {}

for p in os.listdir(noisered_path):
    files = []
    word_blocks = {}
    for f in os.listdir(f"{noisered_path}/{p}"):
        if not f.endswith('.wav') or f.startswith("._"):
            continue
            
        split = f.replace(f"{p}_", "").replace(".wav", "").split("_")
        block = split[0]
        code = split[1]
        mic = split[2]

        code = f"{block}_{code}" if code.startswith("U") else code
        word = fw[code]
        # print(word, block, code, mic)
        if word_blocks.get(word) is None:
            word_blocks[word] = { block : [mic] }
        else:
            if word_blocks[word].get(block) is None:
                word_blocks[word][block] = [mic]
            else:
                word_blocks[word][block].append(mic)
    participants[p] = word_blocks
    # break
# print(participants)

In [10]:
audio = AudioSegment.from_file('/data/zaki/UASpeech/audio/noisereduce/CF02/CF02_B1_UW13_M6.wav')
audio

In [171]:
s = audio.get_array_of_samples()

In [170]:
audio[:400]

In [ ]:
list(zip(range(0,650), s[:650]))

In [161]:
audio[400:1650]

In [ ]:
list(zip(range(400,1650), s[400:1650]))

In [ ]:
orig_folders = {}
normal_folders = {}
participants = {}
for x in os.listdir(orig_path):
    # for f in os.listdir(f"{orig_path}/{x}"):
    #     if f.endswith('.wav') and not f.startswith("."):
            
    orig_folders[x] = [f for f in os.listdir(f"{orig_path}/{x}") if f.endswith('.wav') and not f.startswith("._")]

for x in os.listdir(normal_path):
    normal_folders[x] = [f for f in os.listdir(f"{normal_path}/{x}") if f.endswith('.wav') and not f.startswith("._")]

for x in os.listdir(noisered_path):
    participants[x] = [f for f in os.listdir(f"{noisered_path}/{x}") if f.endswith('.wav') and not f.startswith("._")]

In [179]:
print(len(orig_folders))

28


In [262]:
print(sorted([x for x in orig_folders.keys() if x.startswith("C")]))
print(sorted([x for x in orig_folders.keys() if not x.startswith("C")]))

['CF02', 'CF03', 'CF04', 'CF05', 'CM01', 'CM04', 'CM05', 'CM06', 'CM08', 'CM09', 'CM10', 'CM12', 'CM13']
['F02', 'F03', 'F04', 'F05', 'M01', 'M04', 'M05', 'M07', 'M08', 'M09', 'M10', 'M11', 'M12', 'M14', 'M16']


### Data Distribution

Possibilities of data split:

- Male - Female (different F0)
- Severity
- From each person ()

Additional guidance:
- Severity

Check
- Impact of silence trim to accuracy
- Impact of original, normalized, noise reduction to accuracy. Noise reduction was claimed to have better accurace, per readme.

#### All data

In [ ]:
wc = {}
mwc = {}
fwc = {}
for p,v in participants.items():
    if p.startswith("C"):
        continue
    files = []
    for f in v:
        files.append(f.replace(f"{p}_", "").replace(".wav", ""))
    # for f in sorted(files):
    #     print(f)
    for f in sorted(files):
        split = f.split("_")
        code = "_".join(split[:-1]) if split[1].startswith("UW") else split[1]
        word = fw[code]
    #     if not split[1].startswith("UW"):
    #         print(f)
    # break
        if wc.get(word) is None:
            wc[word] = 1
        else:
            wc[word] = wc[word] + 1
        # print(f, fw.get(code))
        if p.startswith("M"):
            if mwc.get(word) is None:
                mwc[word] = 1
            else:
                mwc[word] = mwc[word] + 1
        if p.startswith("F"):
            if fwc.get(word) is None:
                fwc[word] = 1
            else:
                fwc[word] = fwc[word] + 1
for p in wc.keys():
    print(p, mwc[p], fwc[p])

Check distribution of blocks for each word

In [ ]:
wb = {}
for p,v in participants.items():
    if p.startswith("C"):
        continue
    files = []
    for f in v:
        files.append(f.replace(f"{p}_", "").replace(".wav", ""))
    for f in sorted(files):
        split = f.split("_")
        code = "_".join(split[:-1]) if split[1].startswith("UW") else split[1]
        word = fw[code]
        if wb.get(word) is None:
            wb[word] = [split[0]]
        else:
            wb[word].append(split[0])
for p in wb.keys():
    print(p, set(wb[p]))

All common words, digits, and letters have B1, B2, B3 blocks. All uncommon words have only one of them.
Check distribution of each blocks.

In [ ]:
wbc = {}
for p,v in participants.items():
    if p.startswith("C"):
        continue
    files = []
    
    for f in v:
        files.append(f.replace(f"{p}_", "").replace(".wav", ""))
        
    for f in sorted(files):
        split = f.split("_")
        code = "_".join(split[:-1]) if split[1].startswith("UW") else split[1]
        block = split[0]
        word = fw[code]
        if wb.get(word) is None:
            wb[word] = { block : 1 }
        else:
            if wb[word].get(block) is None:
                wb[word][block] = 1
            else:
                wb[word][block] += 1 
for p in wb.keys():
    print(p, wb[p])
    
# xxx = pd.DataFrame(wb).transpose()
# xxx

Distribution of each blocks looks fine

In [ ]:
wc = {}
mwc = {}
fwc = {}
for p,v in participants.items():
    if p.startswith("C"):
        continue
    files = []
    for f in v:
        files.append(f.replace(f"{p}_", "").replace(".wav", ""))
    # for f in sorted(files):
    #     print(f)
    for f in sorted(files):
        split = f.split("_")
        code = "_".join(split[:-1]) if split[1].startswith("UW") else split[1]
        word = fw[code]
        #     if not split[1].startswith("UW"):
        #         print(f)
        # break
        if wc.get(word) is None:
            wc[word] = 1
        else:
            wc[word] = wc[word] + 1
        # print(f, fw.get(code))
        if p.startswith("M"):
            if mwc.get(word) is None:
                mwc[word] = 1
            else:
                mwc[word] = mwc[word] + 1
        if p.startswith("F"):
            if fwc.get(word) is None:
                fwc[word] = 1
            else:
                fwc[word] = fwc[word] + 1
for p in wc.keys():
    print(p, mwc[p], fwc[p])

### Data Preparations

Take from each person and words 

Extract labels

In [ ]:
word_files = {}
for p,v in participants.items():
    if p.startswith("C"):
        continue
    files = []
    for f in v:
        files.append(f.replace(f"{p}_", "").replace(".wav", ""))
    # for f in sorted(files):
    #     print(f)
    for f in sorted(files):
        split = f.split("_")
        code = "_".join(split[:-1]) if split[1].startswith("UW") else split[1]
        word = fw[code]
        if 

160
Contains speech: False
CF02 None None None
['B3', 'B2', 'B1']
['M5', 'M8', 'M7', 'M6', 'M3', 'M2', 'M4']
CF02 Alpha B1 M2
/data/zaki/UASpeech/audio/noisereduce/CF02/CF02_B1_LA_M2.wav
['B1']
[] None None None
['B3', 'B1', 'B2']
['M2', 'M5', 'M8', 'M3', 'M7', 'M4', 'M6']
F02 Alpha B1 M2
/data/zaki/UASpeech/audio/noisereduce/F02/F02_B1_LA_M2.wav
['B1']
F02 banana B1 M3
/data/zaki/UASpeech/audio/noisereduce/F02/F02_B1_UW94_M3.wav
CF02 banana B1 M3
/data/zaki/UASpeech/audio/noisereduce/CF02/CF02_B1_UW94_M3.wav
F02 banana B1 M5
/data/zaki/UASpeech/audio/noisereduce/F02/F02_B1_UW94_M5.wav
